<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,309 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Fetched 3,159 kB in 2s (1,289 kB/s)
Reading package lists... Do

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
data=[("James","Bond"),("Scott","Varsa")]

In [6]:
df = spark.createDataFrame(data=data,schema=["col1","col2"])

In [7]:
df.show()

+-----+-----+
| col1| col2|
+-----+-----+
|James| Bond|
|Scott|Varsa|
+-----+-----+



In [8]:
from pyspark.sql.functions import expr

In [9]:
df1 = df.withColumn("Name",
              expr("col1 || ',' ||col2"))

In [10]:
df1.show()

+-----+-----+-----------+
| col1| col2|       Name|
+-----+-----+-----------+
|James| Bond| James,Bond|
|Scott|Varsa|Scott,Varsa|
+-----+-----+-----------+



In [11]:
data = [("James","M"),("Michael","F"),("Jen","")]
columns = ["name","gender"]
df2 = spark.createDataFrame(data = data, schema = columns)

In [12]:
df2.show()

+-------+------+
|   name|gender|
+-------+------+
|  James|     M|
|Michael|     F|
|    Jen|      |
+-------+------+



In [15]:
df3 = df2.withColumn("New_gender",
                     expr("CASE WHEN gender = 'M' THEN 'Male' "+
                          "WHEN gender = 'F' THEN 'Female' "+
                          "ELSE 'Unknown' END"))

In [16]:
df3.show()

+-------+------+----------+
|   name|gender|New_gender|
+-------+------+----------+
|  James|     M|      Male|
|Michael|     F|    Female|
|    Jen|      |   Unknown|
+-------+------+----------+



In [19]:
data=[("2019-01-23",1),("2019-06-24",2),("2019-09-20",3)]
df4 =spark.createDataFrame(data).toDF("date","increment")

In [20]:
df4.show()

+----------+---------+
|      date|increment|
+----------+---------+
|2019-01-23|        1|
|2019-06-24|        2|
|2019-09-20|        3|
+----------+---------+



In [24]:
df5 = df4.select(df4.date,df4.increment,
           expr("add_months(date,increment)").alias("date_increment"))

In [25]:
df5.show()

+----------+---------+--------------+
|      date|increment|date_increment|
+----------+---------+--------------+
|2019-01-23|        1|    2019-02-23|
|2019-06-24|        2|    2019-08-24|
|2019-09-20|        3|    2019-12-20|
+----------+---------+--------------+



In [26]:
df6 = df4.select(df4.date,df4.increment,
           expr("""add_months(date,increment) as date_increment"""))

In [27]:
df6.show()

+----------+---------+--------------+
|      date|increment|date_increment|
+----------+---------+--------------+
|2019-01-23|        1|    2019-02-23|
|2019-06-24|        2|    2019-08-24|
|2019-09-20|        3|    2019-12-20|
+----------+---------+--------------+



In [28]:
df7 = df4.select("increment",
                 expr("""cast(increment as string) as str_increment"""))

In [30]:
df7.printSchema()

root
 |-- increment: long (nullable = true)
 |-- str_increment: string (nullable = true)



In [33]:
df8 = df4.select("date","increment",
                 expr("increment+5 as new_increment"))

In [34]:
df8.show()

+----------+---------+-------------+
|      date|increment|new_increment|
+----------+---------+-------------+
|2019-01-23|        1|            6|
|2019-06-24|        2|            7|
|2019-09-20|        3|            8|
+----------+---------+-------------+



In [35]:
data=[(100,2),(200,3000),(500,500)]
df9=spark.createDataFrame(data).toDF("col1","col2")

In [37]:
df9.show()

+----+----+
|col1|col2|
+----+----+
| 100|   2|
| 200|3000|
| 500| 500|
+----+----+



In [38]:
df9.filter(expr("col1 == col2")).show()

+----+----+
|col1|col2|
+----+----+
| 500| 500|
+----+----+

